In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

In [ ]:
file = "horizons_results (2).txt"
# i manually did added "|" characters, so i could mark the column split locations
hdr  = ' Date__(UT)__HR:MN    |R.A._____(ICRF)_____DEC |R.A.__(a-apparent)__DEC |Azi____(a-app)___Elev |X_(sat-primary)_Y|SatPANG  | T-mag  |N-mag  |   Illu% | ang-sep/v |ObsSub-LON|ObsSub-LAT  | NP.ang | NP.dist  |             r|       rdot |           delta|     deldot |   S-T-O   |    TDB-UT| Sky_motion |Sky_mot_PA |RelVel-ANG |Lun_Sky_Brt |sky_SNR'
cols = hdr.split('|')
names = cols

widths = np.zeros(len(cols),dtype=int)
for k in range(len(cols)):
    widths[k] = int(len(cols[k]) + 1)
    names[k] = cols[k].strip() # make a clean version of the column names


# this reads all columns correctly, and skips the header, but doesnt skip the footer
df = pd.read_fwf(file, skiprows=71, names = names, widths=widths)

df = df.iloc[0:21601,:] # had to find this row limit manually to drop the footer

In [ ]:
df['date'] = pd.to_datetime( df['Date__(UT)__HR:MN'].str[0:17] )
df["deldot"] = df["deldot"].astype(float)
df["acceleration"] = df["deldot"].astype(float).diff() / 60

print(df[["date", "deldot", "acceleration"]].head(20))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df["date"], df['deldot'].astype(float), color='blue', label="Line of Sight Velocity")
plt.xlabel("Time (UTC)")
plt.ylabel("Deldot (km/s)")
plt.title("Deldot vs Time")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
observation_time = [
    ("2025-07-03-05:10:38", 78),
    ("2025-07-05-04:50:14", 66),
    ("2025-07-08-04:52:40", 204),
    ("2025-07-09-05:20:16", 90),
    ("2025-07-10-05:16:28", 18),
]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df["date"], df["acceleration"], color='#e41a1c')

for start_time, duration in observation_time:
    start = pd.to_datetime(start_time, format="%Y-%m-%d-%H:%M:%S")
    end = start + timedelta(minutes=duration)
    plt.axvspan(start, end, color='orange', alpha=0.3)

    #Getting the average and standard deviation of acceleration in this time chunk
    data_in_chunk = df[(df["date"] >= start) & (df["date"] < end)]["acceleration"].dropna()
    if not data_in_chunk.empty:
        avg_acceleration = data_in_chunk.mean()
        std_acceleration = data_in_chunk.std()
        print(f"Date: {start}; AVG Acceleration: {avg_acceleration} km/s²; STD Acceleration: {std_acceleration} km/s²")


plt.xlabel("Time (UTC)")
plt.ylabel("Acceleration (km/s²)")
plt.xlim(left=pd.to_datetime("2025-07-02-05:00:00"), right=pd.to_datetime("2025-07-11-05:30:00"))
#plt.title("Acceleration vs Time")
plt.legend()
plt.tight_layout()
plt.show()
